NBA_Box_Scores_2024.csv.

In [ ]:
# Referenced W3schools for some parts of code
import requests
from bs4 import BeautifulSoup, Comment
import pandas as pd
import time
import os

SCHEDULE_FILE = "NBA_Schedule_2024.csv"
OUTPUT_FILE = "NBA_Box_Scores_2024.csv"
REQUIRED_COLS = ["Date", "Visitor", "Home", "Box Score Link"]

OUTPUT_COLUMNS = [
    "Date", "Visitor", "Home",
    "Visitor Q1", "Visitor Q2", "Visitor Q3", "Visitor Q4", "Visitor Total",
    "Home Q1", "Home Q2", "Home Q3", "Home Q4", "Home Total",
    "Visitor Pace", "Visitor eFG%", "Visitor TOV%", "Visitor ORB%", "Visitor FT/FGA", "Visitor ORtg",
    "Home Pace", "Home eFG%", "Home TOV%", "Home ORB%", "Home FT/FGA", "Home ORtg"
]

def extract_table(soup, table_id):
    table = soup.find("table", {"id": table_id})
    if table is None:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for comment in comments:
            if table_id in comment:
                try:
                    comment_soup = BeautifulSoup(comment, "html.parser")
                    table = comment_soup.find("table", {"id": table_id})
                    if table:
                        return table
                except Exception as e:
                    pass
    return table

def parse_line_score(soup):
    visitor_line = ["N/A"] * 5
    home_line = ["N/A"] * 5

    table = extract_table(soup, "line_score")
    if table:
        tbody = table.find("tbody")
        if tbody:
            rows = tbody.find_all("tr")
        else:
            rows = table.find_all("tr")
            
        if len(rows) >= 2:
            visitor_cells = rows[0].find_all(["th", "td"])
            if visitor_cells:
                if visitor_cells[0].name == "th":
                    visitor_cells = visitor_cells[1:]
                visitor_data = [cell.get_text(strip=True) for cell in visitor_cells]
            else:
                visitor_data = []
            
            home_cells = rows[1].find_all(["th", "td"])
            if home_cells:
                if home_cells[0].name == "th":
                    home_cells = home_cells[1:]
                home_data = [cell.get_text(strip=True) for cell in home_cells]
            else:
                home_data = []
            
            if len(visitor_data) >= 5:
                visitor_line = visitor_data[:4] + [visitor_data[-1]] if len(visitor_data) > 5 else visitor_data
            if len(home_data) >= 5:
                home_line = home_data[:4] + [home_data[-1]] if len(home_data) > 5 else home_data

    return visitor_line, home_line

def parse_four_factors(soup):
    visitor_factors = ["N/A"] * 6
    home_factors = ["N/A"] * 6

    table = extract_table(soup, "four_factors")
    if table:
        tbody = table.find("tbody")
        if tbody:
            rows = tbody.find_all("tr")
        else:
            rows = table.find_all("tr")
            
        if len(rows) >= 2:
            visitor_cells = rows[0].find_all(["th", "td"])
            if visitor_cells:
                if visitor_cells[0].name == "th":
                    visitor_cells = visitor_cells[1:]
                visitor_data = [cell.get_text(strip=True) for cell in visitor_cells]
            else:
                visitor_data = []
            
            home_cells = rows[1].find_all(["th", "td"])
            if home_cells:
                if home_cells[0].name == "th":
                    home_cells = home_cells[1:]
                home_data = [cell.get_text(strip=True) for cell in home_cells]
            else:
                home_data = []
            
            if len(visitor_data) >= 6:
                visitor_factors = visitor_data[:5] + [visitor_data[-1]] if len(visitor_data) > 6 else visitor_data
            if len(home_data) >= 6:
                home_factors = home_data[:5] + [home_data[-1]] if len(home_data) > 6 else home_data

    return visitor_factors, home_factors

def scrape_one_game(game_url):
    full_url = "https://www.basketball-reference.com" + game_url
    print("Fetching:", full_url)
    try:
        response = requests.get(full_url, timeout=10, verify=False)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        visitor_line, home_line = parse_line_score(soup)
        visitor_factors, home_factors = parse_four_factors(soup)
        return visitor_line, home_line, visitor_factors, home_factors
    except Exception as e:
        print("Error fetching", full_url, "-", e)
        return None, None, None, None

def main():
    if not os.path.exists(SCHEDULE_FILE):
        print(f"ERROR: {SCHEDULE_FILE} not found.")
        return
    schedule_df = pd.read_csv(SCHEDULE_FILE)
    for col in REQUIRED_COLS:
        if col not in schedule_df.columns:
            print(f"ERROR: Missing column '{col}' in {SCHEDULE_FILE}.")
            return

    output_data = []
    if os.path.exists(OUTPUT_FILE):
        try:
            existing_df = pd.read_csv(OUTPUT_FILE)
            output_data = existing_df.values.tolist()
            print(f"Resuming from {OUTPUT_FILE} with {len(output_data)} records.")
        except Exception as e:
            print(f"Could not load {OUTPUT_FILE}: {e}\nStarting fresh.")

    for idx, row in schedule_df.iterrows():
        date_str = row["Date"]
        visitor_team = row["Visitor"]
        home_team = row["Home"]
        link = row["Box Score Link"]

        if pd.isna(link):
            print(f"Skipping row {idx} because 'Box Score Link' is missing.")
            continue

        visitor_line, home_line, visitor_factors, home_factors = scrape_one_game(link)
        if visitor_line is None:
            print(f"Skipping game: {date_str}, {visitor_team} vs {home_team} (failed).")
            continue

        record = [
            date_str,
            visitor_team,
            home_team,
            *visitor_line,       
            *home_line,           
            *visitor_factors,     
            *home_factors        
        ]
        output_data.append(record)

        temp_df = pd.DataFrame(output_data, columns=OUTPUT_COLUMNS)
        temp_df.to_csv(OUTPUT_FILE, index=False)
        print(f"Saved progress: {len(output_data)} games processed so far.")
        time.sleep(5) 

    print("Done! Final data saved to", OUTPUT_FILE)

if __name__ == "__main__":
    main()

In [ ]:
# Referenced W3schools for some parts of code
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment
import os
from io import StringIO

SCHEDULE_FILE = "NBA_Schedule_2024.csv"
TEAM_ABBREVIATIONS = {
    "Los Angeles Lakers": "LAL",
    "Denver Nuggets": "DEN",
}

def extract_table_from_html(soup, table_id):
    table = soup.find("table", {"id": table_id})
    if table is None:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for comment in comments:
            if table_id in comment:
                try:
                    comment_soup = BeautifulSoup(comment, "html.parser")
                    table = comment_soup.find("table", {"id": table_id})
                    if table:
                        return table
                except Exception:
                    pass
    return table

def get_team_table(soup, team_abbr, table_type):
    ids_to_try = [
        f"box-{team_abbr}-game-{table_type}",
        f"box-{team_abbr.lower()}-game-{table_type}",
        f"box-{team_abbr.upper()}-game-{table_type}"
    ]
    for table_id in ids_to_try:
        table = extract_table_from_html(soup, table_id)
        if table is not None:
            return table
    return None

def parse_team_totals(table_soup):
    if not table_soup:
        return {}
    try:
        df_list = pd.read_html(StringIO(str(table_soup)))
    except Exception as e:
        print("Error reading table HTML:", e)
        return {}
    if not df_list:
        return {}
    df = df_list[0]
    row_mask = df.iloc[:, 0].astype(str).str.strip().str.lower() == "team totals"
    if not row_mask.any():
        row_mask = df.iloc[:, 0].astype(str).str.contains("team totals", case=False, na=False)
    if not row_mask.any():
        return {}
    totals_row = df[row_mask].iloc[0]
    col_names = df.columns.tolist()
    row_vals = totals_row.tolist()
    return dict(zip(col_names, row_vals))

def fetch_basic_advanced_totals(box_url, visitor_abbr, home_abbr):
    full_url = "https://www.basketball-reference.com" + box_url
    print("Fetching Box Score:", full_url)
    try:
        resp = requests.get(full_url, timeout=10, verify=False)
        resp.raise_for_status()
    except Exception as e:
        print("Error fetching box score:", e)
        return {}
    
    soup = BeautifulSoup(resp.text, "html.parser")
    results = {}
    
    vb_table = get_team_table(soup, visitor_abbr, "basic")
    results["visitor_basic"] = parse_team_totals(vb_table)
    
    va_table = get_team_table(soup, visitor_abbr, "advanced")
    results["visitor_advanced"] = parse_team_totals(va_table)
    
    hb_table = get_team_table(soup, home_abbr, "basic")
    results["home_basic"] = parse_team_totals(hb_table)
    
    ha_table = get_team_table(soup, home_abbr, "advanced")
    results["home_advanced"] = parse_team_totals(ha_table)
    
    return results

if not os.path.exists(SCHEDULE_FILE):
    print(f"Schedule file '{SCHEDULE_FILE}' not found!")
else:
    schedule_df = pd.read_csv(SCHEDULE_FILE)
    test_game = schedule_df[schedule_df["Box Score Link"].notna()].iloc[0]
    
    print("Testing Game Details:")
    print("Date:          ", test_game["Date"])
    print("Visitor:       ", test_game["Visitor"])
    print("Home:          ", test_game["Home"])
    print("Box Score Link:", test_game["Box Score Link"])
    
    visitor_abbr = TEAM_ABBREVIATIONS.get(test_game["Visitor"])
    home_abbr = TEAM_ABBREVIATIONS.get(test_game["Home"])
    
    if not visitor_abbr or not home_abbr:
        print("Error: Missing team abbreviation for one or both teams!")
    else:
        totals = fetch_basic_advanced_totals(test_game["Box Score Link"], visitor_abbr, home_abbr)
        print("\n--- Team Totals for This Game ---")
        print("Visitor Basic:   ", totals.get("visitor_basic"))
        print("Visitor Advanced:", totals.get("visitor_advanced"))
        print("Home Basic:      ", totals.get("home_basic"))
        print("Home Advanced:   ", totals.get("home_advanced"))

Output: --- Team Totals for This Game ---
Visitor Basic:    {('Unnamed: 0_level_0', 'Starters'): 'Team Totals', ('Basic Box Score Stats', 'MP'): '240', ('Basic Box Score Stats', 'FG'): '41', ('Basic Box Score Stats', 'FGA'): '90', ('Basic Box Score Stats', 'FG%'): '.456', ('Basic Box Score Stats', '3P'): '10', ('Basic Box Score Stats', '3PA'): '29', ('Basic Box Score Stats', '3P%'): '.345', ('Basic Box Score Stats', 'FT'): '15', ('Basic Box Score Stats', 'FTA'): '20', ('Basic Box Score Stats', 'FT%'): '.750', ('Basic Box Score Stats', 'ORB'): '13', ('Basic Box Score Stats', 'DRB'): '31', ('Basic Box Score Stats', 'TRB'): '44', ('Basic Box Score Stats', 'AST'): '23', ('Basic Box Score Stats', 'STL'): '5', ('Basic Box Score Stats', 'BLK'): '4', ('Basic Box Score Stats', 'TOV'): '12', ('Basic Box Score Stats', 'PF'): '18', ('Basic Box Score Stats', 'PTS'): '107', ('Basic Box Score Stats', 'GmSc'): nan, ('Basic Box Score Stats', '+/-'): nan}
Visitor Advanced: {('Unnamed: 0_level_0', 'Starters'): 'Team Totals', ('Advanced Box Score Stats', 'MP'): '240', ('Advanced Box Score Stats', 'TS%'): '.541', ('Advanced Box Score Stats', 'eFG%'): '.511', ('Advanced Box Score Stats', '3PAr'): '.322', ('Advanced Box Score Stats', 'FTr'): '.222', ('Advanced Box Score Stats', 'ORB%'): '28.3', ('Advanced Box Score Stats', 'DRB%'): '77.5', ('Advanced Box Score Stats', 'TRB%'): '51.2', ('Advanced Box Score Stats', 'AST%'): '56.1', ('Advanced Box Score Stats', 'STL%'): '5.2', ('Advanced Box Score Stats', 'BLK%'): '7.0', ('Advanced Box Score Stats', 'TOV%'): '10.8', ('Advanced Box Score Stats', 'USG%'): '100.0', ('Advanced Box Score Stats', 'ORtg'): '111.1', ('Advanced Box Score Stats', 'DRtg'): '123.6', ('Advanced Box Score Stats', 'BPM'): nan}
Home Basic:       {('Unnamed: 0_level_0', 'Starters'): 'Team Totals', ('Basic Box Score Stats', 'MP'): '240', ('Basic Box Score Stats', 'FG'): '48', ('Basic Box Score Stats', 'FGA'): '91', ('Basic Box Score Stats', 'FG%'): '.527', ('Basic Box Score Stats', '3P'): '14', ('Basic Box Score Stats', '3PA'): '34', ('Basic Box Score Stats', '3P%'): '.412', ('Basic Box Score Stats', 'FT'): '9', ('Basic Box Score Stats', 'FTA'): '12', ('Basic Box Score Stats', 'FT%'): '.750', ('Basic Box Score Stats', 'ORB'): '9', ('Basic Box Score Stats', 'DRB'): '33', ('Basic Box Score Stats', 'TRB'): '42', ('Basic Box Score Stats', 'AST'): '29', ('Basic Box Score Stats', 'STL'): '9', ('Basic Box Score Stats', 'BLK'): '6', ('Basic Box Score Stats', 'TOV'): '12', ('Basic Box Score Stats', 'PF'): '15', ('Basic Box Score Stats', 'PTS'): '119', ('Basic Box Score Stats', 'GmSc'): nan, ('Basic Box Score Stats', '+/-'): nan}
Home Advanced:    {('Unnamed: 0_level_0', 'Starters'): 'Team Totals', ('Advanced Box Score Stats', 'MP'): '240', ('Advanced Box Score Stats', 'TS%'): '.618', ('Advanced Box Score Stats', 'eFG%'): '.604', ('Advanced Box Score Stats', '3PAr'): '.374', ('Advanced Box Score Stats', 'FTr'): '.132', ('Advanced Box Score Stats', 'ORB%'): '22.5', ('Advanced Box Score Stats', 'DRB%'): '71.7', ('Advanced Box Score Stats', 'TRB%'): '48.8', ('Advanced Box Score Stats', 'AST%'): '60.4', ('Advanced Box Score Stats', 'STL%'): '9.3', ('Advanced Box Score Stats', 'BLK%'): '9.8', ('Advanced Box Score Stats', 'TOV%'): '11.1', ('Advanced Box Score Stats', 'USG%'): '100.0', ('Advanced Box Score Stats', 'ORtg'): '123.6', ('Advanced Box Score Stats', 'DRtg'): '111.1', ('Advanced Box Score Stats', 'BPM'): nan}

Basic and Advanced “Team Totals

In [ ]:
# Referenced W3schools for some parts of code
import requests
import pandas as pd
from bs4 import BeautifulSoup, Comment
import os
import time
from io import StringIO

SCHEDULE_FILE = "NBA_Schedule_2024.csv"     
OUTPUT_FILE = "NBA_Team_Totals_2024.csv"       


TEAM_ABBREVIATIONS = {
    "Atlanta Hawks": "ATL",
    "Boston Celtics": "BOS",
    "Brooklyn Nets": "BRK",
    "Charlotte Hornets": "CHO",
    "Chicago Bulls": "CHI",
    "Cleveland Cavaliers": "CLE",
    "Dallas Mavericks": "DAL",
    "Denver Nuggets": "DEN",
    "Detroit Pistons": "DET",
    "Golden State Warriors": "GSW",
    "Houston Rockets": "HOU",
    "Indiana Pacers": "IND",
    "Los Angeles Clippers": "LAC",
    "Los Angeles Lakers": "LAL",
    "Memphis Grizzlies": "MEM",
    "Miami Heat": "MIA",
    "Milwaukee Bucks": "MIL",
    "Minnesota Timberwolves": "MIN",
    "New Orleans Pelicans": "NOP",
    "New York Knicks": "NYK",
    "Oklahoma City Thunder": "OKC",
    "Orlando Magic": "ORL",
    "Philadelphia 76ers": "PHI",
    "Phoenix Suns": "PHO",
    "Portland Trail Blazers": "POR",
    "Sacramento Kings": "SAC",
    "San Antonio Spurs": "SAS",
    "Toronto Raptors": "TOR",
    "Utah Jazz": "UTA",
    "Washington Wizards": "WAS"
}

def extract_table_from_html(soup, table_id):
    table = soup.find("table", {"id": table_id})
    if table is None:
        comments = soup.find_all(string=lambda text: isinstance(text, Comment))
        for comment in comments:
            if table_id in comment:
                try:
                    comment_soup = BeautifulSoup(comment, "html.parser")
                    table = comment_soup.find("table", {"id": table_id})
                    if table:
                        return table
                except Exception:
                    pass
    return table

def parse_team_totals(table_soup):
    if not table_soup:
        return {}
    try:
        df_list = pd.read_html(StringIO(str(table_soup)))
    except Exception as e:
        print("Error reading table HTML:", e)
        return {}
    if not df_list:
        return {}
    df = df_list[0]
    row_mask = df.iloc[:, 0].astype(str).str.strip().str.lower() == "team totals"
    if not row_mask.any():
        row_mask = df.iloc[:, 0].astype(str).str.contains("team totals", case=False, na=False)
    if not row_mask.any():
        return {}
    totals_row = df[row_mask].iloc[0]
    col_names = df.columns.tolist()
    row_vals = totals_row.tolist()
    return dict(zip(col_names, row_vals))

def get_team_table(soup, team_abbr, table_type):
    ids_to_try = [
        f"box-{team_abbr}-game-{table_type}",
        f"box-{team_abbr.lower()}-game-{table_type}",
        f"box-{team_abbr.upper()}-game-{table_type}"
    ]
    for table_id in ids_to_try:
        table = extract_table_from_html(soup, table_id)
        if table is not None:
            return table
    return None

def fetch_basic_advanced_totals(box_url, visitor_abbr, home_abbr):
    full_url = "https://www.basketball-reference.com" + box_url
    print("Fetching Box Score:", full_url)
    try:
        resp = requests.get(full_url, timeout=10, verify=False)
        resp.raise_for_status()
    except Exception as e:
        print("Error fetching box score:", e)
        return {}
    
    soup = BeautifulSoup(resp.text, "html.parser")
    results = {}
    
    vb_table = get_team_table(soup, visitor_abbr, "basic")
    results["visitor_basic"] = parse_team_totals(vb_table)
    
    va_table = get_team_table(soup, visitor_abbr, "advanced")
    results["visitor_advanced"] = parse_team_totals(va_table)
    
    hb_table = get_team_table(soup, home_abbr, "basic")
    results["home_basic"] = parse_team_totals(hb_table)
    
    ha_table = get_team_table(soup, home_abbr, "advanced")
    results["home_advanced"] = parse_team_totals(ha_table)
    
    return results

def flatten_totals_dict(prefix, totals_dict):
    flat = {}
    for key, val in totals_dict.items():
        if isinstance(key, tuple):
            new_key = prefix + "_" + "_".join(str(k) for k in key)
        else:
            new_key = prefix + "_" + str(key)
        flat[new_key] = val
    return flat

def main():
    if not os.path.exists(SCHEDULE_FILE):
        print(f"Schedule file '{SCHEDULE_FILE}' not found!")
        return
    schedule_df = pd.read_csv(SCHEDULE_FILE)
    
    for col in ["Date", "Visitor", "Home", "Box Score Link"]:
        if col not in schedule_df.columns:
            print(f"Schedule file is missing required column: {col}")
            return
    
    all_records = []  
    
    for idx, row in schedule_df.iterrows():
        box_url = row["Box Score Link"]
        if pd.isna(box_url) or not isinstance(box_url, str):
            print(f"Skipping row {idx} due to missing Box Score Link.")
            continue
        
        date_val = row["Date"]
        visitor_name = row["Visitor"]
        home_name = row["Home"]
        
        visitor_abbr = TEAM_ABBREVIATIONS.get(visitor_name)
        home_abbr = TEAM_ABBREVIATIONS.get(home_name)
        if not visitor_abbr or not home_abbr:
            print(f"Skipping game on {date_val} {visitor_name} vs {home_name} due to missing abbreviation.")
            continue
        
        print(f"Processing game: {date_val} - {visitor_name} @ {home_name}")
        totals = fetch_basic_advanced_totals(box_url, visitor_abbr, home_abbr)
        if not totals:
            print("Skipping game due to failed fetch.")
            continue
        
        record = {
            "Date": date_val,
            "Visitor": visitor_name,
            "Home": home_name,
            "Box Score Link": box_url
        }

        for key in ["visitor_basic", "visitor_advanced", "home_basic", "home_advanced"]:
            flat = flatten_totals_dict(key, totals.get(key, {}))
            record.update(flat)
        
        all_records.append(record)
        time.sleep(3)
    
    if not all_records:
        print("No games processed.")
        return
    
    final_df = pd.DataFrame(all_records)
    final_df.to_csv(OUTPUT_FILE, index=False)
    print(f"Done! Wrote data for {len(final_df)} games to '{OUTPUT_FILE}'.")

if __name__ == "__main__":
    main()

Get the Line Score (Team Scores in each quarter), and Four Factors (different game stats)

In [ ]:
# Referenced W3schools for some parts of code
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

try:
    schedule_df = pd.read_csv("NBA_Schedule_2024.csv")  
    print(" NBA_Schedule_2024.csv Loaded Successfully!")
except FileNotFoundError:
    print(" ERROR: NBA_Schedule_2024.csv not found. Ensure the file is in the correct directory.")
    exit()
except pd.errors.EmptyDataError:
    print(" ERROR: NBA_Schedule_2024.csv is empty.")
    exit()
except pd.errors.ParserError:
    print(" ERROR: Issue parsing NBA_Schedule_2024.csv. Check if it's formatted correctly.")
    exit()

expected_columns = ["Date", "Visitor", "Home", "Box Score Link"]
if not all(col in schedule_df.columns for col in expected_columns):
    print(" ERROR: NBA_Schedule_2024.csv is missing required columns.")
    print(f"Expected: {expected_columns}")
    print(f"Found: {list(schedule_df.columns)}")
    exit()


proxy = "43.201.121.81:80"  
proxies = {
    "http": f"http://{proxy}",
    "https": f"http://{proxy}"
}


box_score_base_url = "https://www.basketball-reference.com"
box_score_data = []  

print(" Starting Box Score Scraping...")

for index, row in schedule_df.iterrows():
    if pd.notna(row["Box Score Link"]): 
        game_url = box_score_base_url + row["Box Score Link"]
        print(f" Fetching: {game_url}") 

        for _ in range(3): 
            try:
                response = requests.get(game_url, proxies=proxies, timeout=10, verify=False)  
                response.raise_for_status()  
                break  
            except requests.exceptions.RequestException as e:
                print(f" Warning: {e}")
                time.sleep(10) 
        else:
            print(f"Skipping {game_url} after multiple failed attempts.")
            continue 

        soup = BeautifulSoup(response.text, "html.parser")
        print(f"Successfully fetched: {row['Visitor']} vs {row['Home']} on {row['Date']}")  

        teams = [row["Visitor"], row["Home"]]
        records = {}
        for team in teams:
            team_header = soup.find("a", string=team)
            if team_header and team_header.find_next_sibling():
                records[team] = team_header.find_next_sibling().text.strip()

        line_score_table = soup.find("table", {"id": "line_score"})
        line_scores = {team: [] for team in teams}
        if line_score_table:
            for tr in line_score_table.find_all("tr")[1:]:  
                team_abbr = tr.find("a").text.strip()
                points = [td.text.strip() for td in tr.find_all("td")]
                line_scores[team] = points

        four_factors_table = soup.find("table", {"id": "four_factors"})
        four_factors = {team: [] for team in teams}
        if four_factors_table:
            for tr in four_factors_table.find_all("tr")[1:]:  
                team_abbr = tr.find("a").text.strip()
                stats = [td.text.strip() for td in tr.find_all("td")]
                four_factors[team] = stats

        box_score_data.append([
            row["Date"], row["Visitor"], records.get(row["Visitor"], "N/A"),
            row["Home"], records.get(row["Home"], "N/A"),
            *line_scores[row["Visitor"]], *line_scores[row["Home"]],
            *four_factors[row["Visitor"]], *four_factors[row["Home"]]
        ])
        time.sleep(10) 

print("Completed Box Score Scraping!")

box_score_columns = [
    "Date", "Visitor", "Visitor Record", "Home", "Home Record",
    "Visitor Q1 Pts", "Visitor Q2 Pts", "Visitor Q3 Pts", "Visitor Q4 Pts", "Visitor Total Pts",
    "Home Q1 Pts", "Home Q2 Pts", "Home Q3 Pts", "Home Q4 Pts", "Home Total Pts",
    "Visitor Pace", "Visitor eFG%", "Visitor TOV%", "Visitor ORB%", "Visitor FT/FGA", "Visitor ORtg",
    "Home Pace", "Home eFG%", "Home TOV%", "Home ORB%", "Home FT/FGA", "Home ORtg"
]

if len(box_score_data) > 0:
    box_score_df = pd.DataFrame(box_score_data, columns=box_score_columns)
    box_score_df.to_csv("NBA_Box_Scores_2024.csv", index=False)
    print("NBA_Box_Scores_2024.csv saved!")
else:
    print("No Box Score Data Found!")